<a href="https://colab.research.google.com/github/gauravreddy08/pytorch-vision-transformer/blob/main/vit_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>